# Solar thermal analysis - CDMX

## Packages

In [1]:
using Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("Plots")

using HTTP

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


## Dictionaries used for the analysis

In [2]:
parameters = Dict(
	"hourly" => [
		"PS",
		"WS2M",
		"QV2M",
		"CLRSKY_SFC_SW_DWN",
		"ALLSKY_SFC_SW_DWN",
		"CLOUD_AMT",
		"ALLSKY_SFC_UVA",
		"ALLSKY_SFC_UVB"
	]
);

descriptions = Dict(
	"PS" => [
		"Surface Pressure",
		"The average of surface pressure at the surface of the earth."
	],
	"WS2M" => [
		"Wind Speed at 2 Meters",
		"The average of wind speed at 2 meters above the surface of the earth."
	],
	"QV2M" => [
		"Specific Humidity at 2 Meters",
		"The ratio of the mass of water vapor to the total mass of air at 2 meters (kg water/kg total air)."
	],
	"CLRSKY_SFC_SW_DWN" => [
		"Clear Sky Surface Shortwave Downward Irradiance",
		"""The total solar irradiance incident (direct plus diffuse) on a horizontal plane at the surface of the earth under clear sky conditions. An alternative term for the total solar irradiance is the "Global Horizontal Irradiance" or GHI."""
	],
	"ALLSKY_SFC_SW_DWN" => [
		"All Sky Surface Shortwave Downward Irradiance",
		"""The total solar irradiance incident (direct plus diffuse) on a horizontal plane at the surface of the earth under all sky conditions. An alternative term for the total solar irradiance is the "Global Horizontal Irradiance" or GHI."""
	],
	"CLOUD_AMT" => [
		"Cloud Amount",
		"The average percent of cloud amount during the temporal period."
	],
	"ALLSKY_SFC_UVA" => [
		"All Sky Surface UVA Irradiance",
		"The ultraviolet A (UVA 315nm-400nm) irradiance under all sky conditions."
	],
	"ALLSKY_SFC_UVB" => [
		"All Sky Surface UVB Irradiance",
		"The ultraviolet B (UVB 280nm-315nm) irradiance under all sky conditions."
	]
);

In [3]:
const LATITUDE::Float64 = 19.5138;
const LONGITUDE::Float64 = -99.1285;
const START_DATE::Int64 = 20220101;
const END_DATE::Int64 = 20221231;


In [7]:
requests = []
for key in keys(parameters)
	URL::String = string(
		"https://power.larc.nasa.gov/api/temporal/$key/point",
		"?parameters=",
		join(parameters[key], ","),
		"&community=RE",
		"&longitude=$LONGITUDE",
		"&latitude=$LATITUDE",
		"&start=$START_DATE",
		"&end=$END_DATE",
		"&format=CSV"
	);
	# res = HTTP.request("GET", URL)
	# push!(requests, res)
	display(URL)
end

"https://power.larc.nasa.gov/api/temporal/hourly/point?parameters=PS,WS2M,QV2M,CLRSKY_SFC_SW_DWN,ALLSKY_SFC_SW_DWN,CLOUD_AMT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB&community=RE&longitude=-99.1285&latitude=19.5138&start=20220101&end=20221231&format=CSV"